<a href="https://colab.research.google.com/github/ashishagg70/ContextBasedQA/blob/master/ContextBasedQAV1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!wget "https://data.deepai.org/squad1.1.zip"

--2021-04-18 17:15:50--  https://data.deepai.org/squad1.1.zip
Resolving data.deepai.org (data.deepai.org)... 138.201.36.183
Connecting to data.deepai.org (data.deepai.org)|138.201.36.183|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 9152254 (8.7M) [application/x-zip-compressed]
Saving to: ‘squad1.1.zip.1’

squad1.1.zip.1      100%[===================>]   8.73M  4.10MB/s    in 2.1s    

2021-04-18 17:15:54 (4.10 MB/s) - ‘squad1.1.zip.1’ saved [9152254/9152254]



In [ ]:
!unzip squad1.1.zip

Archive:  squad1.1.zip
replace dev-v1.1.json? [y]es, [n]o, [A]ll, [N]one, [r]ename: N


In [ ]:
import numpy as np
from collections import defaultdict
from nltk.tokenize import word_tokenize
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchtext.vocab import GloVe
import json

import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
file = open('train-v1.1.json','r')
file2 = open('dev-v1.1.json','r')
train_data = json.load(file)['data']
dev_data = json.load(file2)['data']

In [ ]:
MAX_WORD_LENGTH = 40
D = 100

In [ ]:
glove = GloVe('6B',100)

In [ ]:
char_to_index = defaultdict(int)
index_to_char = defaultdict(str)
word_to_index = defaultdict(int)
index_to_word = defaultdict(str)

num_contexts = 0
num_questions = 0
num_characters = 0
num_words = 0

char_contexts = []
char_questions = []

word_contexts=[]
word_questions = []
answers = []

input_data = []

In [ ]:
def char_encode(text, is_train=True):
    global char_to_index
    global index_to_char
    global num_characters

    text_endcoding = []
    for word in text:
        encoding = np.zeros(MAX_WORD_LENGTH)
        try:
            i=0
            for char in word:
                encode = char_to_index[char]
                if is_train == True and encode == 0:
                    index = num_characters+1
                    char_to_index[char]= index
                    index_to_char[index]= char
                    encode = index
                    num_characters+=1
                encoding[i]=encode
                i+=1
        except:
            print(word)
        text_endcoding.append(encoding)
    return text_endcoding

In [ ]:
char_encode(['this','is','good'])

[array([1., 2., 3., 4., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0.]),
 array([3., 4., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0.]),
 array([5., 6., 6., 7., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0.])]

In [ ]:
def convert_to_lower(text):
    return text.lower()

def perform_word_tokenization(text):
    return word_tokenize(text)

def word_encode(text, is_train=True):
    global word_to_index
    global index_to_word
    global num_words
    encoding =[]
    for word in text:
        encode = word_to_index[word]
        if is_train == True and encode == 0:
            index = num_words+1
            word_to_index[word] = index
            index_to_word[index] = word
            encode = index
            num_words+=1
        encoding.append(encode)
    return encoding


In [ ]:
def text_preprocess(text, is_train=True):
    text = convert_to_lower(text)
    text = perform_word_tokenization(text)
    word_encoding = word_encode(text, is_train)
    char_encoding = char_encode(text, is_train)
    return char_encoding, word_encoding

def get_answer_end_index(start,answer):
    return start+len(answer)

In [ ]:
for obj in train_data:
    for para in obj['paragraphs']:
        cce, cwe = text_preprocess(para['context'])
        char_contexts.append(cce)
        word_contexts.append(cwe)
        num_contexts+=1
        for qa in para['qas']:
            qce, qwe = text_preprocess(qa['question'])
            char_questions.append(qce)
            word_questions.append(qwe)
            num_questions+=1
            input_data.append((num_contexts,num_questions))
            ans = [] 
            for a in qa['answers']:
                ans.append((a['answer_start'],get_answer_end_index(a['answer_start'],a['text'])))
            answers.append(ans)

In [ ]:
char_vocab_size = len(char_to_index)+1
word_vocab_size = len(word_to_index)+1

In [ ]:
class Char_Embedding(nn.Module):

    def __init__(self,vocab_size,embedding_dim = 8, cnn_kernel_size = 5,word_embedding_size =100 ):
        super().__init__()
        self.embedding = nn.Embedding(num_embeddings=vocab_size,embedding_dim=embedding_dim,padding_idx=0)
        self.cnn = nn.Conv1d(in_channels=embedding_dim,out_channels=word_embedding_size,kernel_size=cnn_kernel_size)
        self.maxpool = nn.MaxPool1d(kernel_size = MAX_WORD_LENGTH-cnn_kernel_size+1)
    
    def forward(self,x):
        # print("start",x.shape)
        batch_size = x.shape[0]
        x = x.view(-1,MAX_WORD_LENGTH)
        x = self.embedding(x)
        x = x.transpose(1,2)
        x = self.cnn(x)
        # print("cnn",x.shape)
        x = self.maxpool(x)
        # print("pool",x.shape)
        #TODO : ReLU 
        x = x.view(batch_size,-1,x.shape[1])
        # print("final",x.shape)
        return x

In [ ]:
class Word_Embedding(nn.Module):

    def __init__(self):
        super().__init__()      
        emb_matrix = torch.zeros(word_vocab_size,100)
        for word, idx in word_to_index.items():
            try:
                emb_matrix[idx, :] = embedding[word]
            except:
                pass
        self.embedding = nn.Embedding.from_pretrained(emb_matrix)

    def forward(self,x):
        return self.embedding(x)

In [ ]:
class Contextual_Embedding(nn.Module):
    def __init__(self):
        super().__init__()
        self.bilstm = nn.LSTM(2*D,D,1,bidirectional = True,batch_first = True)

    def forward(self,char_embedding,word_embedding):

        x = torch.cat((char_embedding,word_embedding),dim = -1)

        x,_ = self.bilstm(x)
        return x

In [ ]:
class AttentionFlowLayer(nn.Module):
    def __init__(self):
        super().__init__()
        self.alpha = nn.Linear(6*D,1)
    
    def forward(self,H,U):
        # H : contextual embedding of context
        # U : contextual embedding of query

        T = H.shape[1]
        J = U.shape[1]

        H_interleaved = torch.repeat_interleave(H,J,dim=1)
        U_repeated = U.repeat(1,T,1)

        assert(H_interleaved.shape==U_repeated.shape)

        HU = torch.cat((H_interleaved,U_repeated,H_interleaved*U_repeated),dim=-1)
        S = self.alpha(HU)
        S = S.view(-1,T,J)
        C2Q_att = F.softmax(S,dim = -1)
        U_tilde = torch.matmul(C2Q_att,U)
        Q2C_att = F.softmax(torch.max(S,dim=-1)[0],dim=-1)
        Q2C_att = Q2C_att.unsqueeze(1)
        H_tilde = torch.matmul(Q2C_att,H).repeat(1,T,1)
        G = torch.cat((H,U_tilde,H*U_tilde,H*H_tilde),dim=-1)
        return G


In [ ]:
class ModellingLayer(nn.Module):
    def __init__(self):
        super().__init__()
        self.bilstm = nn.LSTM(8*D,D,2,bidirectional = True,batch_first = True)
    def forward(self,G):
        M,_ = self.bilstm(G)
        return M

In [ ]:
class OutputLayer(nn.Module):
    def __init__(self):
        super().__init__()
        self.dense1 = nn.Linear(10*D,1)
        self.dense2 = nn.Linear(10*D,1)
        self.bilstm = nn.LSTM(2*D,D,1,bidirectional = True,batch_first = True)
    def forward(self,G,M):
        GM = torch.cat((G,M),dim=-1)
        start = F.log_softmax(self.dense1(GM),dim=-1)
        M2,_ = self.bilstm(M)
        GM2= torch.cat((G,M2),dim=-1)
        end = F.log_softmax(self.dense1(GM2),dim=-1)

        return start.squeeze(-1),end.squeeze(-1)

In [ ]:
class BiDAF(nn.Module):
    def __init__(self):
        super().__init__()
        self.char_emb_layer=Char_Embedding(char_vocab_size,embedding_dim=8)
        self.word_emb_layer=Word_Embedding()
        self.cont_emb_layer=Contextual_Embedding()
        self.att_layer = AttentionFlowLayer()
        self.modelling_layer = ModellingLayer()
        self.output_layer = OutputLayer()
    
    def forward(self,context_char,context_word,query_char,query_word):
        context_char_emb = self.char_emb_layer(char_batch_context_t)
        context_word_emb = self.word_emb_layer(word_batch_context_t)
        context_cont_emb = self.cont_emb_layer(context_char_emb,context_word_emb)

        query_char_emb = self.char_emb_layer(char_batch_query_t)
        query_word_emb = self.word_emb_layer(word_batch_query_t)
        query_cont_emb = self.cont_emb_layer(query_char_emb,query_word_emb)

        g =self.att_layer(context_cont_emb,query_cont_emb)
        m = modelling(g)
        o = output_layer(g,m)
        return o

In [ ]:
char_batch_context = [char_contexts[0],char_contexts[0]]
char_batch_query = [char_questions[0],char_questions[1]]
word_batch_context=[word_contexts[0],word_contexts[0]]
word_batch_query = [word_questions[0],word_questions[1]]

In [ ]:
char_batch_context_t = torch.LongTensor(nn.utils.rnn.pad_sequence([torch.LongTensor(sent) for sent in char_batch_context],batch_first=True))
char_batch_query_t = torch.LongTensor(nn.utils.rnn.pad_sequence([torch.LongTensor(sent) for sent in char_batch_query],batch_first=True))
word_batch_context_t = torch.LongTensor(nn.utils.rnn.pad_sequence([torch.LongTensor(sent) for sent in word_batch_context],batch_first=True))
word_batch_query_t = torch.LongTensor(nn.utils.rnn.pad_sequence([torch.LongTensor(sent) for sent in word_batch_query],batch_first=True))

In [ ]:
bidaf = BiDAF()

In [ ]:
o = bidaf(char_batch_context_t,word_batch_context_t,char_batch_query_t,word_batch_query_t)

In [ ]:
tot = 0
for param in bidaf.parameters():
    if param.requires_grad==True:
        tot+=param.numel()

In [ ]:
tot

1625003

In [ ]:
bidaf

BiDAF(
  (char_emb_layer): Char_Embedding(
    (embedding): Embedding(1259, 100, padding_idx=0)
    (cnn): Conv1d(100, 100, kernel_size=(5,), stride=(1,))
    (maxpool): MaxPool1d(kernel_size=36, stride=36, padding=0, dilation=1, ceil_mode=False)
  )
  (word_emb_layer): Word_Embedding(
    (embedding): Embedding(103079, 100)
  )
  (cont_emb_layer): Contextual_Embedding(
    (bilstm): LSTM(200, 100, batch_first=True, bidirectional=True)
  )
  (att_layer): AttentionFlowLayer(
    (alpha): Linear(in_features=600, out_features=1, bias=True)
  )
  (modelling_layer): ModellingLayer(
    (bilstm): LSTM(800, 100, num_layers=2, batch_first=True, bidirectional=True)
  )
  (output_layer): OutputLayer(
    (dense1): Linear(in_features=1000, out_features=1, bias=True)
    (dense2): Linear(in_features=1000, out_features=1, bias=True)
    (bilstm): LSTM(200, 100, batch_first=True, bidirectional=True)
  )
)